In [1]:
# Paquetería
# Herramientas matemáticas
import numpy as np

# Lectura de .csv a Dataframes
import pandas as pd

# Herramienta para hacer copias de Dataframes sin conflictos
import copy

import sys
###############################################################
#####                       ¡WARNING!                       ###
############ Para asegurar compatibilidad entre          ######
############  Sistemas operativos comentar Linux      #########
############################################################### 
# LINUX:
dot = '.'
# WINDOWS:
#dot = ''
sys.path.append(dot+'./utils/')
output_path = dot+'./output/'
from utils import *

# 0) Lectura de Vuelos_v1.csv

In [2]:
Vuelos_v2 = pd.read_csv(output_path+'Vuelos_v1.csv')
Vuelos_v2.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0


# 1) Horario
¿En qué horario del día ocurre el vuelo?
- madrugada
- mañana
- tarde
- noche

In [3]:
# Convertimos las columnas a datetime
Vuelos_v2['STA'] = pd.to_datetime(Vuelos_v2['STA'])
Vuelos_v2['STD'] = pd.to_datetime(Vuelos_v2['STD'])

In [4]:
Vuelos_v2['Horario'] =  Vuelos_v2.apply(lambda row : horario(row['STD'].hour),axis=1)
Vuelos_v2.head() 

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Horario
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,1
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN,2
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,1


# 2) Periodo
¿En qué periodo ocurre el vuelo?
- Ordinario
- Vacacional

In [5]:
Vuelos_v2['Periodo'] = Vuelos_v2.apply(lambda row : periodo_col(row['STD']),axis=1)
Vuelos_v2.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Horario,Periodo
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,1,0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,0,0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN,2,0
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2,0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,1,0


# 3) Tiempo de vuelo
¿Cuál es el tiempo de vuelo en minutos?
- Se toman en cuenta sólo vuelos menores a 6 horas

In [6]:
# Agregamos la columna de Tiempo de vuelo
Vuelos_v2['TiempoVuelo'] = Vuelos_v2['STA'] -Vuelos_v2['STD']
Vuelos_v2.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Horario,Periodo,TiempoVuelo
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,1,0,0 days 02:45:00
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,0,0,0 days 04:00:00
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN,2,0,0 days 00:45:00
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2,0,0 days 00:40:00
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,1,0,0 days 01:10:00


In [7]:
# Hacemos el drop de los vuelos que duran más de 6 horas
# Pueden ocurrir por desvíos, retrasos, outliers, etc..
Vuelos_v3 = copy.deepcopy(Vuelos_v2[Vuelos_v2['TiempoVuelo']<'0 days 6:00:00'])
len(Vuelos_v3)

244747

In [8]:
# Convertimos la columna de duración a float
Vuelos_v3['TiempoVuelo'] = Vuelos_v3['TiempoVuelo'].dt.total_seconds() /60

In [9]:
Vuelos_v3.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Horario,Periodo,TiempoVuelo
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,1,0,165.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,0,0,240.0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN,2,0,45.0
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2,0,40.0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,1,0,70.0


# 4) Día de la semana
- Un número entero que representa el día de la semana

In [10]:
#Agregar columna dia de la semana
Vuelos_v3['Dia_Semana'] = pd.to_datetime(Vuelos_v3['STD']).dt.day_name()
Vuelos_v3 = mapear_y_agregar_columna(Vuelos_v3, 'Dia_Semana', 'Mapeo_Dia_Semana')

# 5) Tipo de destino

- Un número entero que indica el tipo de destino

In [11]:
# Mapeo de Destino
Vuelos_v3 = mapear_y_agregar_columna(Vuelos_v3, 'Destination_Type', 'Mapeo_Destino')
Vuelos_v3.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Horario,Periodo,TiempoVuelo,Dia_Semana,Mapeo_Dia_Semana,Mapeo_Destination_Type
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,1,0,165.0,Thursday,0,0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,0,0,240.0,Monday,1,1
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN,2,0,45.0,Friday,2,1
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2,0,40.0,Monday,1,1
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,1,0,70.0,Friday,2,2


# 6) Tipo de origen
- Un número entero que indica el tipo de destino

In [12]:
# Mapeo de Origen
Vuelos_v3 = mapear_y_agregar_columna(Vuelos_v3, 'Origin_Type', 'Mapeo_Origen')

In [13]:
Vuelos_v3.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Horario,Periodo,TiempoVuelo,Dia_Semana,Mapeo_Dia_Semana,Mapeo_Destination_Type,Mapeo_Origin_Type
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,1,0,165.0,Thursday,0,0,0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,0,0,240.0,Monday,1,1,1
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN,2,0,45.0,Friday,2,1,0
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2,0,40.0,Monday,1,1,0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,1,0,70.0,Friday,2,2,0


# Vuelos 2023

In [14]:
# Separamos los vuelos para 2023
Vuelos_2023 = Vuelos_v3[(Vuelos_v3['Passengers'].notnull()) | (Vuelos_v3['Bookings'].notnull())]
Vuelos_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 121620 entries, 0 to 245750
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Flight_ID               121620 non-null  object        
 1   Aeronave                121120 non-null  object        
 2   DepartureStation        121619 non-null  object        
 3   ArrivalStation          121619 non-null  object        
 4   Destination_Type        121619 non-null  object        
 5   Origin_Type             121619 non-null  object        
 6   STD                     121620 non-null  datetime64[ns]
 7   STA                     121620 non-null  datetime64[ns]
 8   Capacity                121620 non-null  int64         
 9   Passengers              121620 non-null  float64       
 10  Bookings                121620 non-null  float64       
 11  Horario                 121620 non-null  int64         
 12  Periodo                 121620 non-

In [15]:
print('La fecha Departure del primer vuelo en 2023: ',Vuelos_2023['STD'].min())
print('La fecha Departure del último vuelo en 2023: ',Vuelos_2023['STD'].max())
print('La fecha Arrival del primer vuelo en 2023: ',Vuelos_2023['STA'].min())
print('La fecha Arrival del último vuelo en 2023: ',Vuelos_2023['STA'].max())

La fecha Departure del primer vuelo en 2023:  2023-01-02 00:20:00
La fecha Departure del último vuelo en 2023:  2023-12-31 23:50:00
La fecha Arrival del primer vuelo en 2023:  2023-01-02 02:45:00
La fecha Arrival del último vuelo en 2023:  2024-01-01 05:15:00


## a) Drop nulls

In [16]:
# Eliminamos los nulls de la base datos
Vuelos_2023 = Vuelos_2023.dropna(how='any')
Vuelos_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 121118 entries, 0 to 245750
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Flight_ID               121118 non-null  object        
 1   Aeronave                121118 non-null  object        
 2   DepartureStation        121118 non-null  object        
 3   ArrivalStation          121118 non-null  object        
 4   Destination_Type        121118 non-null  object        
 5   Origin_Type             121118 non-null  object        
 6   STD                     121118 non-null  datetime64[ns]
 7   STA                     121118 non-null  datetime64[ns]
 8   Capacity                121118 non-null  int64         
 9   Passengers              121118 non-null  float64       
 10  Bookings                121118 non-null  float64       
 11  Horario                 121118 non-null  int64         
 12  Periodo                 121118 non-

## b) Drop outliers
- vuelos mayores a 6 horas

In [17]:
# ¿Algún vuelo dura más de un día?
# Hacemos el drop de los vuelos que duran más de 6 horas = 360 minutos
# Pueden ocurrir por desvíos, retrasos, outliers, etc..
Vuelos_2023_v1 = copy.deepcopy(Vuelos_2023[Vuelos_2023['TiempoVuelo']<=360])
len(Vuelos_2023_v1)

121118

## c) Drop repeated ID's

In [18]:
# Se borran los duplicados de los ID's de vuelo
Vuelos_2023_v1.drop_duplicates(subset=['Flight_ID'], inplace=True)
len(Vuelos_2023_v1)

118545

## d) Incluir la variable de AbordajeReal
- Esta varible toma los pasajeros que estuvieron en el vuelo

In [19]:
# Sobreventa de vuelos
len(Vuelos_2023_v1[Vuelos_2023_v1['Passengers'] > Vuelos_2023_v1['Capacity'] ])

30011

In [20]:
# Copiamos el dataset Vuelos_2023
Vuelos_2023_v2 = copy.deepcopy(Vuelos_2023_v1)

In [21]:
abordaje_real = [filter_row(Vuelos_2023_v1.iloc[i,8],Vuelos_2023_v1.iloc[i,9]) for i in range(len(Vuelos_2023_v1))]
# Agregamos la columna de abordaje_real
Vuelos_2023_v2['AbordajeReal'] = abordaje_real

Vuelos_2023_v2.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Horario,Periodo,TiempoVuelo,Dia_Semana,Mapeo_Dia_Semana,Mapeo_Destination_Type,Mapeo_Origin_Type,AbordajeReal
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,1,0,165.0,Thursday,0,0,0,229.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,0,0,240.0,Monday,1,1,1,186.0
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2,0,40.0,Monday,1,1,0,200.0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,1,0,70.0,Friday,2,2,0,162.0
5,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,125.0,2,0,55.0,Thursday,0,2,2,183.0


# Exportamos datos 2023 para Modelación

In [28]:
Vuelos_2023_v2.to_csv(output_path+'Vuelos_2023_v2.csv',index=False)

# Vuelos 2024

In [23]:
# Las columnas en Nan ocurren en la misma proporción en ambas columnas
Vuelos_2024 = Vuelos_v3[(Vuelos_v3['Passengers'].isnull()) & (Vuelos_v3['Bookings'].isnull()) &(Vuelos_v3['STD']<'2025-01-01 00:00:00')]
Vuelos_2024.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Horario,Periodo,TiempoVuelo,Dia_Semana,Mapeo_Dia_Semana,Mapeo_Destination_Type,Mapeo_Origin_Type
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN,2,0,45.0,Friday,2,1,0
19,bb319ee8f62b5ae38e6f6da4dbff03cb,XA-VAV,BA,AB,Playa,Ciudad Principal,2024-01-12 08:20:00,2024-01-12 09:20:00,186,NaN,NaN,1,0,60.0,Friday,2,2,0
22,d77570d39116f81dbb619c5225dd0572,NaN,AK,BO,MX Amigos y Familia,Playa,2024-05-18 14:10:00,2024-05-18 16:15:00,180,NaN,NaN,2,0,125.0,Saturday,6,1,3
25,85d3b182431d323b856e50e249689981,XA-VIY,AT,AI,Playa,Ciudad Principal,2024-03-18 10:30:00,2024-03-18 13:25:00,186,NaN,NaN,1,0,175.0,Monday,1,2,0
26,04e83b220f5513bd6cc636784dcddec6,XA-VXC,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2024-04-05 00:50:00,2024-04-05 04:45:00,240,NaN,NaN,0,1,235.0,Friday,2,1,1


In [24]:
print('La fecha Departure del primer vuelo en 2024: ',Vuelos_2024['STD'].min())
print('La fecha Departure del último vuelo en 2024: ',Vuelos_2024['STD'].max())
print('La fecha Arrival del primer vuelo en 2024: ',Vuelos_2024['STA'].min())
print('La fecha Arrival del último vuelo en 2024: ',Vuelos_2024['STA'].max())

La fecha Departure del primer vuelo en 2024:  2024-01-01 00:05:00
La fecha Departure del último vuelo en 2024:  2024-12-31 21:10:00
La fecha Arrival del primer vuelo en 2024:  2024-01-01 05:05:00
La fecha Arrival del último vuelo en 2024:  2024-12-31 22:40:00


## a) Drop outliers

In [25]:
# ¿Algún vuelo dura más de un día?
# Hacemos el drop de los vuelos que duran más de 6 horas = 360 minutos
# Pueden ocurrir por desvíos, retrasos, outliers, etc..
Vuelos_2024_v1 = copy.deepcopy(Vuelos_2024[Vuelos_2024['TiempoVuelo']<=360])
len(Vuelos_2024_v1)

121432

## b) Drop repeated ID's

In [26]:
# Se borran los duplicados de los ID's de vuelo
Vuelos_2024_v1.drop_duplicates(subset=['Flight_ID'], inplace=True)
len(Vuelos_2024_v1)

116350

# Exportamos 2024 para test

In [29]:
Vuelos_2024_v1.to_csv(output_path+'Vuelos_2024_v2.csv',index=False)